In [28]:
import math
import os
import pickle
import random

import nltk
import numpy as np
import pandas as pd

In [29]:
nltk.data.path.append('.')

In [30]:
HOME = os.environ['HOME']
punkt_path = f'{HOME}/nltk_data/tokenizers/punkt/PY3/english.pickle'

In [31]:
with open(punkt_path, 'rb') as f:
    tokenizer = pickle.load(f)

## Part 1: Load and Preprocess Data
### Part 1.1: Load the data
You will use twitter data.
Load the data and view the first few sentences by running the next cell.

Notice that data is a long string that contains many many tweets.
Observe that there is a line break "\n" between tweets.

In [8]:
DATA = '../../../../data'

In [9]:
with open (f'{DATA}/en_US.twitter.txt', 'r') as f:
    data = f.read()

In [11]:
print('Data type:', type(data))
print('Length:', len(data))
print('First 300 chars of the data')
print('-------')
print(data[:300])
print('-------')
display(data[:300])
print('-------')
print('Last 300 chars of the data')
print('-------')
display(data[-300:])
print('-------')

Data type: <class 'str'>
Length: 3335477
First 300 chars of the data
-------
How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more fun if I don't.
So Tired D; Played Lazer Tag & Ran A 
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 chars of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


### Part 1.2 Pre-process the data

Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
1. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
1. Assign sentences into train or test sets.
1. Find tokens that appear at least N times in the training data.
1. Replace tokens that appear less than N times by `<unk>`


Note: we omit validation data in this exercise.
- In real applications, we should hold a part of data as a validation set and use it to tune our training.
- We skip this process for simplicity.

In [16]:
def split_to_sentences(data):
    """
    Split data by linebreak "\n"
    
    Args:
        data: str
    
    Returns:
        A list of sentences
    """
    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]    
    return sentences    

In [17]:
# test your code
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)

split_to_sentences(x)


I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

### Exercise 02
The next step is to tokenize sentences (split a sentence into a list of words). 
- Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
- Append each tokenized list of words into a list of tokenized sentences.

In [32]:
def tokenize_sentences(sentences):
    """
    Tokenize sentences into tokens (words)
    
    Args:
        sentences: List of strings
    
    Returns:
        List of lists of tokens
    """
    tokenized_sentences = [nltk.word_tokenize(sent.lower()) 
                           for sent in sentences]
    return tokenized_sentences

In [33]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

### Exercise 03


Use the two functions that you have just implemented to get the tokenized data.
- split the data into sentences
- tokenize those sentences

In [34]:
def get_tokenized_data(data):
    """
    Make a list of tokenized sentences
    
    Args:
        data: String
    
    Returns:
        List of lists of tokens
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    
    # Get the sentences by splitting up the data
    sentences = split_to_sentences(data)
    
    # Get the list of lists of tokens by tokenizing the sentences
    tokenized_sentences = tokenize_sentences(sentences)
    
    ### END CODE HERE ###
    
    return tokenized_sentences

In [35]:
# test
x = 'Sky is blue.\nLeaves are green\nRoses are red.'
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

### Split into train and test sets

Now run the cell below to split data into training and test sets.

In [36]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [37]:
print(f'{len(tokenized_data)} data are split into {len(train_data)} train'
      f'and {len(test_data)} test set')
print('First training sample:')
print(train_data[0])
print('First test sample')
print(test_data[0])

47961 data are split into 38368 trainand 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


### Exercise 04

You won't use all the tokens (words) appearing in the data for training.  Instead, you will use the more frequently used words.  
- You will focus on the words that appear at least N times in the data.
- First count how many times each word appears in the data.

You will need a double for-loop, one for sentences and the other for tokens within a sentence.

In [42]:
def count_words(tokenized_sentences):
    """
    Count the number of word appearence in the tokenized sentences
    
    Args:
        tokenized_sentences: List of lists of strings
    
    Returns:
        dict that maps word (str) to the frequency (int)
    """
    word_counts = {}
    for sentence in tokenized_sentences:
        for token in sentence:
            word_counts[token] = word_counts.get(token, 0) + 1
    return word_counts

In [43]:
# Test
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}